# 如何添加聊天历史


:::note

本教程之前使用 [RunnableWithMessageHistory](https://api.js.langchain.com/classes/_langchain_core.runnables.RunnableWithMessageHistory.html) 构建了一个聊天机器人。你可以在 [v0.2 文档](https://js.langchain.com/v0.2/docs/how_to/qa_chat_history_how_to/) 中访问本教程的这个版本。

与 `RunnableWithMessageHistory` 相比，LangGraph 实现提供了多项优势，包括能够持久化应用程序状态的任意组件（而不仅仅是消息）。

:::

在许多问答应用中，我们希望用户可以进行来回对话，这意味着应用需要对过去的问题和答案具有某种“记忆”，并且需要一些逻辑来将这些信息纳入当前的思考中。

在本指南中，我们重点介绍**添加用于整合历史消息的逻辑**。

这在很大程度上是 [对话式 RAG 教程](/docs/tutorials/qa_chat_history) 的一个精简版本。

我们将介绍两种方法：

1. [链（Chains）](/docs/how_to/qa_chat_history_how_to#chains)，其中我们始终执行检索步骤；
2. [代理（Agents）](/docs/how_to/qa_chat_history_how_to#agents)，其中我们赋予 LLM 决定是否以及如何执行检索步骤（或多个步骤）的权限。

对于外部知识源，我们将使用与 [RAG 教程](/docs/tutorials/rag) 中相同的由 Lilian Weng 撰写的博客文章 [LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/)。

## 环境配置
### 依赖项

在本次演示中，我们将使用一个 OpenAI 的聊天模型和嵌入模型，以及一个 Memory 向量存储。但这里展示的内容适用于任何 [ChatModel](/docs/concepts/chat_models) 或 [LLM](/docs/concepts/text_llms)、[Embeddings](/docs/concepts/embedding_models)、[VectorStore](/docs/concepts/vectorstores) 或 [Retriever](/docs/concepts/retrievers)。

我们将使用以下软件包：

```bash
npm install --save langchain @langchain/openai langchain cheerio uuid
```

我们需要设置环境变量 `OPENAI_API_KEY`：

```bash
export OPENAI_API_KEY=YOUR_KEY
```

### LangSmith

使用LangChain构建的许多应用程序将包含多个步骤，并多次调用LLM。随着这些应用程序变得越来越复杂，能够检查链或代理内部确切发生的情况变得至关重要。最好的方法是使用[LangSmith](https://docs.smith.langchain.com)。

请注意，LangSmith不是必需的，但它非常有帮助。如果您确实想使用LangSmith，在上面的链接注册后，请确保设置您的环境变量以开始记录跟踪信息：


```bash
export LANGSMITH_TRACING=true
export LANGSMITH_API_KEY=您的密钥

# 如果您不在无服务器环境中，请减少跟踪延迟
# export LANGCHAIN_CALLBACKS_BACKGROUND=true
```

## 链 {#chains}

在一个对话式RAG应用中，发送给检索器的查询应该基于对话的上下文。LangChain提供了一个[createHistoryAwareRetriever](https://api.js.langchain.com/functions/langchain.chains_history_aware_retriever.createHistoryAwareRetriever.html)构造函数来简化这一过程。它构建了一个接受键 `input` 和 `chat_history` 作为输入的链，并且输出模式与检索器相同。`createHistoryAwareRetriever` 需要以下输入：

1. LLM；
2. 检索器；
3. 提示词。

首先，我们获取这些对象：

### LLM

我们可以使用任何支持的聊天模型：

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs"

<ChatModelTabs customVarName="llm" />
```

In [1]:
// @lc-docs-hide-cell
import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({ model: "gpt-4o" });

### 初始设置

In [2]:
import { CheerioWebBaseLoader } from "@langchain/community/document_loaders/web/cheerio";
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
import { MemoryVectorStore } from "langchain/vectorstores/memory"
import { OpenAIEmbeddings } from "@langchain/openai";

const loader = new CheerioWebBaseLoader(
  "https://lilianweng.github.io/posts/2023-06-23-agent/"
);

const docs = await loader.load();

const textSplitter = new RecursiveCharacterTextSplitter({ chunkSize: 1000, chunkOverlap: 200 });
const splits = await textSplitter.splitDocuments(docs);
const vectorStore = await MemoryVectorStore.fromDocuments(splits, new OpenAIEmbeddings());

// Retrieve and generate using the relevant snippets of the blog.
const retriever = vectorStore.asRetriever();

### 提示词

我们将使用一个提示词，其中包含一个名为 "chat_history" 的 `MessagesPlaceholder` 变量。这允许我们使用 "chat_history" 输入键向提示词中传入一条消息列表，这些消息将会被插入到系统消息之后，以及包含最新问题的人类消息之前。

In [3]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";

const contextualizeQSystemPrompt = (
  "Given a chat history and the latest user question " +
  "which might reference context in the chat history, " +
  "formulate a standalone question which can be understood " +
  "without the chat history. Do NOT answer the question, " +
  "just reformulate it if needed and otherwise return it as is."
)

const contextualizeQPrompt = ChatPromptTemplate.fromMessages(
  [
    ["system", contextualizeQSystemPrompt],
    new MessagesPlaceholder("chat_history"),
    ["human", "{input}"],
  ]
)

### 组装链

然后我们可以实例化具备历史感知能力的检索器：

In [5]:
import { createHistoryAwareRetriever } from "langchain/chains/history_aware_retriever";

const historyAwareRetriever = await createHistoryAwareRetriever({
  llm,
  retriever,
  rephrasePrompt: contextualizeQPrompt
});


此链会在检索器前添加对输入查询的改写，以便检索过程能够结合对话的上下文。

现在我们可以构建完整的问答链。

与[RAG教程](/docs/tutorials/rag)中一样，我们将使用[createStuffDocumentsChain](https://api.js.langchain.com/functions/langchain.chains_combine_documents.createStuffDocumentsChain.html)生成一个`questionAnswerChain`，其输入键为`context`、`chat_history`和`input`——它接受检索到的上下文以及对话历史和查询以生成答案。

我们使用[createRetrievalChain](https://api.js.langchain.com/functions/langchain.chains_retrieval.createRetrievalChain.html)构建最终的`ragChain`。该链依次应用`historyAwareRetriever`和`questionAnswerChain`，并保留中间输出（如检索到的上下文）以方便使用。它的输入键为`input`和`chat_history`，输出包含`input`、`chat_history`、`context`和`answer`。

In [7]:
import { createStuffDocumentsChain } from "langchain/chains/combine_documents";
import { createRetrievalChain } from "langchain/chains/retrieval";

const systemPrompt = 
  "You are an assistant for question-answering tasks. " +
  "Use the following pieces of retrieved context to answer " +
  "the question. If you don't know the answer, say that you " +
  "don't know. Use three sentences maximum and keep the " +
  "answer concise." +
  "\n\n" +
  "{context}";

const qaPrompt = ChatPromptTemplate.fromMessages([
  ["system", systemPrompt],
  new MessagesPlaceholder("chat_history"),
  ["human", "{input}"],
]);

const questionAnswerChain = await createStuffDocumentsChain({
  llm,
  prompt: qaPrompt,
});

const ragChain = await createRetrievalChain({
  retriever: historyAwareRetriever,
  combineDocsChain: questionAnswerChain,
});

### 聊天历史的有状态管理

我们已经添加了用于整合聊天历史的应用逻辑，但目前仍需手动在整个应用中进行传递。在生产环境中，我们的问答应用通常会将聊天历史持久化到数据库中，并能够适当地读取和更新它。

[LangGraph](https://langchain-ai.github.io/langgraphjs/) 实现了一个内置的 [持久化层](https://langchain-ai.github.io/langgraphjs/concepts/persistence/)，使其非常适合支持多轮对话的聊天应用。

将我们的聊天模型封装在一个最小的 LangGraph 应用中，可以让我们自动持久化消息历史，从而简化多轮对话应用的开发。

LangGraph 配备了一个简单的 [内存检查点存储器](https://langchain-ai.github.io/langgraphjs/reference/classes/checkpoint.MemorySaver.html)，我们下面将使用它。有关更多细节（包括如何使用不同的持久化后端，例如 SQLite 或 Postgres），请参阅其文档。

如需详细了解如何管理消息历史，请前往《如何添加消息历史（记忆）》指南。

In [8]:
import { AIMessage, BaseMessage, HumanMessage } from "@langchain/core/messages";
import { StateGraph, START, END, MemorySaver, messagesStateReducer, Annotation } from "@langchain/langgraph";

// Define the State interface
const GraphAnnotation = Annotation.Root({
  input: Annotation<string>(),
  chat_history: Annotation<BaseMessage[]>({
    reducer: messagesStateReducer,
    default: () => [],
  }),
  context: Annotation<string>(),
  answer: Annotation<string>(),
})

// Define the call_model function
async function callModel(state: typeof GraphAnnotation.State) {
  const response = await ragChain.invoke(state);
  return {
    chat_history: [
      new HumanMessage(state.input),
      new AIMessage(response.answer),
    ],
    context: response.context,
    answer: response.answer,
  };
}

// Create the workflow
const workflow = new StateGraph(GraphAnnotation)
  .addNode("model", callModel)
  .addEdge(START, "model")
  .addEdge("model", END);

// Compile the graph with a checkpointer object
const memory = new MemorySaver();
const app = workflow.compile({ checkpointer: memory });

In [9]:
import { v4 as uuidv4 } from "uuid";

const threadId = uuidv4();
const config = { configurable: { thread_id: threadId } };

const result = await app.invoke(
  { input: "What is Task Decomposition?" },
  config,
)
console.log(result.answer);

Task Decomposition is the process of breaking down a complicated task into smaller, simpler, and more manageable steps. Techniques like Chain of Thought (CoT) and Tree of Thoughts expand on this by enabling agents to think step by step or explore multiple reasoning possibilities at each step. This allows for a more structured and interpretable approach to handling complex tasks.


In [10]:
const result2 = await app.invoke(
  { input: "What is one way of doing it?" },
  config,
)
console.log(result2.answer);

One way of doing task decomposition is by using an LLM with simple prompting, such as asking "Steps for XYZ.\n1." or "What are the subgoals for achieving XYZ?" This method leverages direct prompts to guide the model in breaking down tasks.


可以通过应用程序的状态检查对话历史记录：

In [11]:
const chatHistory = (await app.getState(config)).values.chat_history;
for (const message of chatHistory) {
  console.log(message);
}

HumanMessage {
  "content": "What is Task Decomposition?",
  "additional_kwargs": {},
  "response_metadata": {}
}
AIMessage {
  "content": "Task Decomposition is the process of breaking down a complicated task into smaller, simpler, and more manageable steps. Techniques like Chain of Thought (CoT) and Tree of Thoughts expand on this by enabling agents to think step by step or explore multiple reasoning possibilities at each step. This allows for a more structured and interpretable approach to handling complex tasks.",
  "additional_kwargs": {},
  "response_metadata": {},
  "tool_calls": [],
  "invalid_tool_calls": []
}
HumanMessage {
  "content": "What is one way of doing it?",
  "additional_kwargs": {},
  "response_metadata": {}
}
AIMessage {
  "content": "One way of doing task decomposition is by using an LLM with simple prompting, such as asking \"Steps for XYZ.\\n1.\" or \"What are the subgoals for achieving XYZ?\" This method leverages direct prompts to guide the model in breaking

### 综合运用

![](../../static/img/conversational_retrieval_chain.png)

为方便起见，我们将所有必要步骤整合到一个代码单元格中：

In [12]:
import { CheerioWebBaseLoader } from "@langchain/community/document_loaders/web/cheerio";
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
import { MemoryVectorStore } from "langchain/vectorstores/memory"
import { OpenAIEmbeddings, ChatOpenAI } from "@langchain/openai";
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { createHistoryAwareRetriever } from "langchain/chains/history_aware_retriever";
import { createStuffDocumentsChain } from "langchain/chains/combine_documents";
import { createRetrievalChain } from "langchain/chains/retrieval";
import { AIMessage, BaseMessage, HumanMessage } from "@langchain/core/messages";
import { StateGraph, START, END, MemorySaver, messagesStateReducer, Annotation } from "@langchain/langgraph";
import { v4 as uuidv4 } from "uuid";

const llm2 = new ChatOpenAI({ model: "gpt-4o" });

const loader2 = new CheerioWebBaseLoader(
  "https://lilianweng.github.io/posts/2023-06-23-agent/"
);

const docs2 = await loader2.load();

const textSplitter2 = new RecursiveCharacterTextSplitter({ chunkSize: 1000, chunkOverlap: 200 });
const splits2 = await textSplitter2.splitDocuments(docs2);
const vectorStore2 = await MemoryVectorStore.fromDocuments(splits2, new OpenAIEmbeddings());

// Retrieve and generate using the relevant snippets of the blog.
const retriever2 = vectorStore2.asRetriever();

const contextualizeQSystemPrompt2 =
  "Given a chat history and the latest user question " +
  "which might reference context in the chat history, " +
  "formulate a standalone question which can be understood " +
  "without the chat history. Do NOT answer the question, " +
  "just reformulate it if needed and otherwise return it as is.";

const contextualizeQPrompt2 = ChatPromptTemplate.fromMessages(
  [
    ["system", contextualizeQSystemPrompt2],
    new MessagesPlaceholder("chat_history"),
    ["human", "{input}"],
  ]
)

const historyAwareRetriever2 = await createHistoryAwareRetriever({
  llm: llm2,
  retriever: retriever2,
  rephrasePrompt: contextualizeQPrompt2
});

const systemPrompt2 = 
  "You are an assistant for question-answering tasks. " +
  "Use the following pieces of retrieved context to answer " +
  "the question. If you don't know the answer, say that you " +
  "don't know. Use three sentences maximum and keep the " +
  "answer concise." +
  "\n\n" +
  "{context}";

const qaPrompt2 = ChatPromptTemplate.fromMessages([
  ["system", systemPrompt2],
  new MessagesPlaceholder("chat_history"),
  ["human", "{input}"],
]);

const questionAnswerChain2 = await createStuffDocumentsChain({
  llm: llm2,
  prompt: qaPrompt2,
});

const ragChain2 = await createRetrievalChain({
  retriever: historyAwareRetriever2,
  combineDocsChain: questionAnswerChain2,
});

// Define the State interface
const GraphAnnotation2 = Annotation.Root({
  input: Annotation<string>(),
  chat_history: Annotation<BaseMessage[]>({
    reducer: messagesStateReducer,
    default: () => [],
  }),
  context: Annotation<string>(),
  answer: Annotation<string>(),
})

// Define the call_model function
async function callModel2(state: typeof GraphAnnotation2.State) {
  const response = await ragChain2.invoke(state);
  return {
    chat_history: [
      new HumanMessage(state.input),
      new AIMessage(response.answer),
    ],
    context: response.context,
    answer: response.answer,
  };
}

// Create the workflow
const workflow2 = new StateGraph(GraphAnnotation2)
  .addNode("model", callModel2)
  .addEdge(START, "model")
  .addEdge("model", END);

// Compile the graph with a checkpointer object
const memory2 = new MemorySaver();
const app2 = workflow2.compile({ checkpointer: memory2 });

const threadId2 = uuidv4();
const config2 = { configurable: { thread_id: threadId2 } };

const result3 = await app2.invoke(
  { input: "What is Task Decomposition?" },
  config2,
)
console.log(result3.answer);

const result4 = await app2.invoke(
  { input: "What is one way of doing it?" },
  config2,
)
console.log(result4.answer);

Task Decomposition is the process of breaking a complicated task into smaller, simpler steps to enhance model performance on complex tasks. Techniques like Chain of Thought (CoT) and Tree of Thoughts (ToT) are used for this, with CoT focusing on step-by-step thinking and ToT exploring multiple reasoning possibilities at each step. Decomposition can be carried out by the LLM itself, using task-specific instructions, or through human inputs.
One way of doing task decomposition is by prompting the LLM with simple instructions such as "Steps for XYZ.\n1." or "What are the subgoals for achieving XYZ?" This encourages the model to break down the task into smaller, manageable steps on its own.


## 代理 {#agents}

代理利用LLM的推理能力在执行过程中做出决策。使用代理可以让我们将检索过程的部分判断权下放。尽管它们的行为比链（chains）更难预测，但在此背景下它们具有一些优势：
- 代理直接生成检索器的输入，而无需我们像上面那样显式地构建上下文化内容；
- 代理可以为了满足查询执行多个检索步骤，或者完全不执行任何检索步骤（例如，响应用户的通用问候语）。

### 检索工具

代理可以访问“工具”并管理其执行。在这种情况下，我们将把检索器转换为LangChain工具，供代理使用：

In [13]:
import { createRetrieverTool } from "langchain/tools/retriever";

const tool =  createRetrieverTool(
    retriever,
    {
      name: "blog_post_retriever",
      description: "Searches and returns excerpts from the Autonomous Agents blog post.",
    }
)
const tools = [tool]

### 代理构造函数

既然我们已经定义了工具和LLM，现在可以创建代理了。我们将使用[LangGraph](https://langchain-ai.github.io/langgraphjs)来构建代理。
目前我们使用的是一个高级接口来构建代理，但LangGraph的优点在于，这个高级接口背后有一个低级的、高度可控的API，以防你想修改代理的逻辑。

In [16]:
import { createReactAgent } from "@langchain/langgraph/prebuilt";

const agentExecutor = createReactAgent({ llm, tools })

我们现在可以尝试一下。请注意，目前它还不是有状态的（我们还需要添加内存支持）

In [17]:
const query = "What is Task Decomposition?"

for await (const s of await agentExecutor.stream(
  { messages: [{ role: "user", content: query }] },
)){
  console.log(s)
  console.log("----")
}

{
  agent: {
    messages: [
      AIMessage {
        "id": "chatcmpl-AB7xlcJBGSKSp1GvgDY9FP8KvXxwB",
        "content": "",
        "additional_kwargs": {
          "tool_calls": [
            {
              "id": "call_Ev0nA6nzGwOeMC5upJUUxTuw",
              "type": "function",
              "function": "[Object]"
            }
          ]
        },
        "response_metadata": {
          "tokenUsage": {
            "completionTokens": 19,
            "promptTokens": 66,
            "totalTokens": 85
          },
          "finish_reason": "tool_calls",
          "system_fingerprint": "fp_52a7f40b0b"
        },
        "tool_calls": [
          {
            "name": "blog_post_retriever",
            "args": {
              "query": "Task Decomposition"
            },
            "type": "tool_call",
            "id": "call_Ev0nA6nzGwOeMC5upJUUxTuw"
          }
        ],
        "invalid_tool_calls": [],
        "usage_metadata": {
          "input_tokens": 66,
          "outpu

LangGraph 自带内置持久化功能，因此我们无需使用 `ChatMessageHistory`！相反，我们可以直接向 LangGraph 代理传入一个检查点器。

不同的对话通过在配置对象中为对话线程指定一个键来管理，如下所示。

In [19]:
import { MemorySaver } from "@langchain/langgraph";

const memory3 = new MemorySaver();

const agentExecutor2 = createReactAgent({ llm, tools, checkpointSaver: memory3 })

这就是我们构建一个对话式RAG代理所需的所有内容。

让我们观察它的行为。请注意，如果我们输入一个不需要检索步骤的查询，代理将不会执行检索步骤：

In [20]:
const threadId3 = uuidv4();
const config3 = { configurable: { thread_id: threadId3 } };

for await (const s of await agentExecutor2.stream({ messages: [{ role: "user", content: "Hi! I'm bob" }] }, config3)) {
  console.log(s)
  console.log("----")
}

{
  agent: {
    messages: [
      AIMessage {
        "id": "chatcmpl-AB7y8P8AGHkxOwKpwMc3qj6r0skYr",
        "content": "Hello, Bob! How can I assist you today?",
        "additional_kwargs": {},
        "response_metadata": {
          "tokenUsage": {
            "completionTokens": 12,
            "promptTokens": 64,
            "totalTokens": 76
          },
          "finish_reason": "stop",
          "system_fingerprint": "fp_e375328146"
        },
        "tool_calls": [],
        "invalid_tool_calls": [],
        "usage_metadata": {
          "input_tokens": 64,
          "output_tokens": 12,
          "total_tokens": 76
        }
      }
    ]
  }
}
----


此外，如果我们输入一个需要检索步骤的查询，代理将生成工具的输入：

In [21]:
const query2 = "What is Task Decomposition?"

for await (const s of await agentExecutor2.stream({ messages: [{ role: "user", content: query2 }] }, config3)) {
  console.log(s)
  console.log("----")
}

{
  agent: {
    messages: [
      AIMessage {
        "id": "chatcmpl-AB7y8Do2IHJ2rnUvvMU3pTggmuZud",
        "content": "",
        "additional_kwargs": {
          "tool_calls": [
            {
              "id": "call_3tSaOZ3xdKY4miIJdvBMR80V",
              "type": "function",
              "function": "[Object]"
            }
          ]
        },
        "response_metadata": {
          "tokenUsage": {
            "completionTokens": 19,
            "promptTokens": 89,
            "totalTokens": 108
          },
          "finish_reason": "tool_calls",
          "system_fingerprint": "fp_e375328146"
        },
        "tool_calls": [
          {
            "name": "blog_post_retriever",
            "args": {
              "query": "Task Decomposition"
            },
            "type": "tool_call",
            "id": "call_3tSaOZ3xdKY4miIJdvBMR80V"
          }
        ],
        "invalid_tool_calls": [],
        "usage_metadata": {
          "input_tokens": 89,
          "outp

在上面的例子中，代理并没有将我们的查询逐字插入到工具中，而是去除了诸如"what"和"is"这样的不必要的词语。

同样的原理使得代理在必要时可以利用对话的上下文：

In [22]:
const query3 = "What according to the blog post are common ways of doing it? redo the search"

for await (const s of await agentExecutor2.stream({ messages: [{ role: "user", content: query3 }] }, config3)) {
  console.log(s)
  console.log("----")
}

{
  agent: {
    messages: [
      AIMessage {
        "id": "chatcmpl-AB7yDE4rCOXTPZ3595GknUgVzASmt",
        "content": "",
        "additional_kwargs": {
          "tool_calls": [
            {
              "id": "call_cWnDZq2aloVtMB4KjZlTxHmZ",
              "type": "function",
              "function": "[Object]"
            }
          ]
        },
        "response_metadata": {
          "tokenUsage": {
            "completionTokens": 21,
            "promptTokens": 1089,
            "totalTokens": 1110
          },
          "finish_reason": "tool_calls",
          "system_fingerprint": "fp_52a7f40b0b"
        },
        "tool_calls": [
          {
            "name": "blog_post_retriever",
            "args": {
              "query": "common ways of task decomposition"
            },
            "type": "tool_call",
            "id": "call_cWnDZq2aloVtMB4KjZlTxHmZ"
          }
        ],
        "invalid_tool_calls": [],
        "usage_metadata": {
          "input_tokens": 1

请注意，智能体能够推断出我们查询中的“it”指的是“任务分解”，并因此生成了一个合理的搜索查询——在本例中是“常见的任务分解方法”。

### 将它们整合在一起

为方便起见，我们将所有必要步骤整合到一个代码单元格中：

In [23]:
import { createRetrieverTool } from "langchain/tools/retriever";
import { createReactAgent } from "@langchain/langgraph/prebuilt";
import { MemorySaver } from "@langchain/langgraph";
import { ChatOpenAI } from "@langchain/openai";
import { CheerioWebBaseLoader } from "@langchain/community/document_loaders/web/cheerio";
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
import { MemoryVectorStore } from "langchain/vectorstores/memory"
import { OpenAIEmbeddings } from "@langchain/openai";

const llm3 = new ChatOpenAI({ model: "gpt-4o" });

const loader3 = new CheerioWebBaseLoader(
  "https://lilianweng.github.io/posts/2023-06-23-agent/"
);

const docs3 = await loader3.load();

const textSplitter3 = new RecursiveCharacterTextSplitter({ chunkSize: 1000, chunkOverlap: 200 });
const splits3 = await textSplitter3.splitDocuments(docs3);
const vectorStore3 = await MemoryVectorStore.fromDocuments(splits3, new OpenAIEmbeddings());

// Retrieve and generate using the relevant snippets of the blog.
const retriever3 = vectorStore3.asRetriever();

const tool2 = createRetrieverTool(
    retriever3,
    {
      name: "blog_post_retriever",
      description: "Searches and returns excerpts from the Autonomous Agents blog post.",
    }
)
const tools2 = [tool2]
const memory4 = new MemorySaver();

const agentExecutor3 = createReactAgent({ llm: llm3, tools: tools2, checkpointSaver: memory4 })

## 下一步

我们已经介绍了构建基本对话式问答应用程序的步骤：

- 我们使用链（chains）构建了一个可预测的应用程序，该程序为每个用户输入生成搜索查询；
- 我们使用代理（agents）构建了一个应用程序，该应用程序会“决定”何时以及如何生成搜索查询。

如需探索不同类型的检索器（retrievers）和检索策略，请访问指南中的 [检索器](/docs/how_to#retrievers) 部分。

如需详细了解 LangChain 的对话记忆抽象功能，请访问 [如何添加消息历史记录（记忆）](/docs/how_to/message_history) LCEL 页面。
